In [1]:
# %load_ext iminizinc
%load_ext autoreload
%autoreload 2

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
from decimal import *
getcontext().prec = 6
import seaborn as sns
from scipy.signal import find_peaks, peak_widths
import pickle
import re


import sys, os
sys.path.append(os.path.abspath('..'))
from research.pkg.lib import *


# import mplfinance as mpf
%matplotlib inline

# plt.rcParams['lines.linewidth'] = 0.5 



In [2]:

# Machine learning libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import Ridge, Lasso, LinearRegression


from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [3]:
pair = 'BTC_USDT'
timeframe = '4h'

exchange = 'binance'
odf = pd.read_json(f'../freq-user-data/data/{exchange}/{pair}-{timeframe}.json'
# exchange = 'kucoin'
# odf = pd.read_json(f'../../freq-user-data/data/{exchange}/futures/{pair}-{timeframe}-futures.json'
).dropna().set_axis(['timestamp', 'open', 'high', 'low', 'close', 'volume'], axis=1
).assign(dtime=lambda x: pd.to_datetime(x['timestamp'], unit='ms', utc=False)
).set_index('dtime').sort_index()

print(odf.shape)
odf.head(4)

(12676, 6)


,timestamp,open,high,low,close,volume
dtime,,,,,,
2017-08-17 04:00:00,1502942400000,4261.48,4349.99,4261.32,4349.99,82.088865
2017-08-17 08:00:00,1502956800000,4333.32,4485.39,4333.32,4427.30,63.619882
2017-08-17 12:00:00,1502971200000,4436.06,4485.39,4333.42,4352.34,174.562001
2017-08-17 16:00:00,1502985600000,4352.33,4354.84,4200.74,4325.23,225.109716


In [5]:
### Load it all. Period.

idf = odf.copy().ffill()

# body low and highs
idf['body_high'] = idf[['open', 'close']].max(axis=1)
idf['body_low'] = idf[['open', 'close']].min(axis=1)

# log return
idf['lret'] = idf.close.divide(idf.close.shift()).apply(np.log)
# candle price direction, up or down (1,-1)
idf['candle_dir'] = 1 - 2*(idf.lret < 0)
# wicks log percent form candle
idf['wiLdist'] = idf.low.divide(idf.body_low).apply(np.log)
idf['wiHdist'] = idf.high.divide(idf.body_high).apply(np.log)
# number of down candles in last n periods
for per in [2,4,6,12]:
    idf[f'n_down_{per}'] = (idf.candle_dir < 0).astype(int).rolling(per).sum().abs()
    idf[f'n_up_{per}'] = (idf.candle_dir > 0).astype(int).rolling(per).sum().abs()
# future returns for close and max/min high/low
future_periods = [1,2,3,6]
for fu in future_periods:
    idf[f'flretC_{fu}'] = idf.close.shift(-fu).divide(idf.close).apply(np.log)
    idf[f'flretH_{fu}'] = idf.high.rolling(fu).max().shift(-fu).divide(idf.close).apply(np.log)
    idf[f'flretL_{fu}'] = idf.high.rolling(fu).max().shift(-fu).divide(idf.close).apply(np.log)

idf.to_csv('test.csv')
idf = idf.dropna()
idf

,timestamp,open,high,low,close,volume,body_high,body_low,lret,candle_dir,...,flretL_1,flretC_2,flretH_2,flretL_2,flretC_3,flretH_3,flretL_3,flretC_6,flretH_6,flretL_6
dtime,,,,,,,,,,,,,,,,,,,,,
2017-08-19 00:00:00,1503100800000,4108.37,4184.69,4084.28,4138.55,108.510448,4138.55,4108.37,0.007319,1,...,0.004208,-0.044708,0.004208,0.004208,-0.041063,0.004208,0.004208,-0.010672,0.004208,0.004208
2017-08-19 04:00:00,1503115200000,4138.50,4156.00,3933.21,4033.47,112.752079,4138.50,4033.47,-0.025718,-1,...,0.012021,-0.015345,0.016360,0.016360,0.010519,0.017316,0.017316,0.029895,0.030061,0.030061
2017-08-19 08:00:00,1503129600000,4033.47,4082.25,3850.00,3957.60,77.256042,4033.47,3957.60,-0.018989,-1,...,0.035349,0.029508,0.036305,0.036305,0.045053,0.047468,0.047468,0.036941,0.062081,0.062081
2017-08-19 12:00:00,1503144000000,3945.12,4100.00,3928.89,3972.05,47.515578,3972.05,3945.12,0.003645,1,...,0.032660,0.041409,0.043824,0.043824,0.030392,0.043824,0.043824,0.037784,0.058437,0.058437
2017-08-19 16:00:00,1503158400000,4036.19,4103.92,3953.40,4076.12,25.925056,4076.12,4036.19,0.025863,1,...,0.017960,0.004528,0.017960,0.017960,0.019376,0.019542,0.019542,0.021475,0.032574,0.032574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-31 16:00:00,1685548800000,26911.55,27123.43,26839.01,27010.57,5607.896990,27010.57,26911.55,0.003673,1,...,0.010658,-0.008575,0.012488,0.012488,-0.007727,0.012488,0.012488,-0.005490,0.012488,0.012488
2023-05-31 20:00:00,1685563200000,27010.57,27299.99,27005.16,27210.35,3962.300610,27210.35,27010.57,0.007369,1,...,0.005119,-0.015096,0.005119,0.005119,-0.011733,0.005119,0.005119,-0.014527,0.005119,0.005119
2023-06-01 00:00:00,1685577600000,27210.36,27350.00,26605.05,26779.94,11213.187950,27210.36,26779.94,-0.015944,-1,...,0.003863,0.004211,0.006370,0.006370,0.004268,0.007492,0.007492,0.008184,0.014644,0.014644


In [6]:
(idf.candle_dir < 0).astype(int)#.rolling(per).sum().abs()

dtime
2017-08-19 00:00:00    0
2017-08-19 04:00:00    1
2017-08-19 08:00:00    1
2017-08-19 12:00:00    0
2017-08-19 16:00:00    0
                      ..
2023-05-31 16:00:00    0
2023-05-31 20:00:00    0
2023-06-01 00:00:00    1
2023-06-01 04:00:00    0
2023-06-01 08:00:00    0
Name: candle_dir, Length: 12659, dtype: int64